# Create a planar robot

In [1]:
from datetime import datetime
from pathlib import Path

import numpy as np
import pinocchio as pin

from timor.Bodies import Body, Connector, Gender
from timor.Joints import Joint
from timor.Module import AtomicModule, ModuleAssembly, ModulesDB, ModuleHeader
from timor.Geometry import Box, ComposedGeometry, Cylinder, Sphere
from timor.utilities.transformation import Transformation
from timor.utilities.spatial import rotX, rotY, rotZ

2023-06-29 18:17:55,694 Timor INFO Loading custom configurations from /home/lukas/Dokumente/Uni/MasterRCI/Praktikum_RFL_for_RM/code/ident/timor-python/timor.config
2023-06-29 18:17:55,796 Timor INFO Skipping PROMODULAR that has already been downloaded.
2023-06-29 18:17:55,796 Timor INFO Skipping modrob-gen2 that has already been downloaded.


## The following code is taken from the create_some_modules tutorial, so you should have gone through that first to understand what's going on here

In [2]:
connector_size = 1  # Doesn't matter as long as it is the same for all connectors
ROT_X = Transformation(rotX(np.pi))

In [3]:
def base() -> ModulesDB:
    """
    Creates a base connector attached to an empty body. The output connector is pointing in positive x-direction to keep it basically planar.
    """
    l = 0  # No geometry
    geometry = Box({'x': l, 'y': l, 'z': l}, pose=Transformation.from_translation([0, 0, l / 2]))
    c_world = Connector('base', ROT_X, gender=Gender.f, connector_type='base', size=connector_size)
    # How to orient the connector?
    # - It has to "point away" from our module
    # - The z-axis of the connector should be aligned with the global x-axis
    # - We assume the body coordinate system to be aligned with the global coordinate system for the base, so:
    c_robot = Connector('base2robot', gender=Gender.m, connector_type='default', size=connector_size,
                        body2connector=Transformation.from_translation([l / 2, 0, l / 2]) @ rotY(np.pi / 2) @ rotZ(np.pi))  # The rotZ is just "cosmetics"
    return ModulesDB({AtomicModule(ModuleHeader(ID='base', name='Base'),[Body('base', collision=geometry, connectors=[c_world, c_robot])])})


def eef() -> ModulesDB:
    """Creates an end effector that's basically just a small sphere (nice for visualization)."""
    diameter = .1
    geometry = Sphere({'r': diameter}, pose=Transformation.from_translation([0, 0, diameter / 2]))
    c_robot = Connector('robot2eef', ROT_X, gender=Gender.f, connector_type='default', size=connector_size)
    c_world = Connector('end-effector', gender=Gender.m, connector_type='eef',
                        body2connector=Transformation.from_translation([0, 0, diameter / 2]))
    return ModulesDB({AtomicModule(ModuleHeader(ID='eef', name='Demo EEF'), [Body('EEF', collision=geometry, connectors=[c_robot, c_world])])})


def i_links(sizes) -> ModulesDB:
    """Create I links with a certain lengths."""
    links = ModulesDB()
    diameter = .1
    
    for size in sizes:
        module_header = ModuleHeader(ID='i_{}'.format(int(size * 100)), name='I shaped link {}'.format(int(size * 100)))
        connectors = (
            Connector(f'{int(diameter * 100)}-{i}',
                      ROT_X @ Transformation.from_translation([0, 0, size / 2]) if i == 0
                      else Transformation.from_translation([0, 0, size / 2]),
                      gender=Gender.f if i == 0 else Gender.m,
                      connector_type='default',
                      size=connector_size)
            for i in range(2)
        )
        geometry = Cylinder({'r': diameter / 2, 'z': size}, pose=Transformation.from_translation([0, 0, 0]))
        body = Body('i_{}'.format(int(size * 100)), collision=geometry, connectors=connectors)
        links.add(AtomicModule(module_header, [body]))
    return links

## Create a planar joint
This joint "lives" in the xy-plane, so input and output stay in the plane why the rotation happens around the z-axis.

This is an "abstract" module with no geometric expansion

In [4]:
def joint() -> ModulesDB:
    length = 0
    proximal = Body('J1_proximal', collision=Cylinder({'r': 0, 'z': 0}),
                    connectors=[Connector('J1_proximal', ROT_X, gender=Gender.f, connector_type='default', size=connector_size)]
                    )
    distal = Body('J1_distal', collision=Cylinder({'r': 0, 'z': 0}),
                  connectors=[Connector('J1_distal', Transformation.neutral(), gender=Gender.m, connector_type='default', size=connector_size)],
                  )
    joint = Joint(
        joint_id='joint',
        joint_type='revolute',
        parent_body=proximal,
        child_body=distal,
        q_limits=[-np.pi, np.pi],  # This is arbitrary
        parent2joint=rotY(-np.pi/2),  # Make the rotation axis the global z-Axis
        joint2child=rotY(np.pi/2)  # Rotate back to the plane we are "living" in
    )

    module_header = ModuleHeader(ID='J1',name='Prismatic Joint')
    return ModulesDB({
        AtomicModule(module_header, [proximal, distal], [joint])
    })

In [5]:
db = base().union(eef()).union(i_links([1, 2, 5])).union(joint())
print("You defined the following modules:", ', '.join(db.by_id))

You defined the following modules: i_200, J1, i_100, base, i_500, eef


In [6]:
assembly = ModuleAssembly.from_serial_modules(db, ('base', 'i_200', 'J1', 'i_100'))
trajectory = np.linspace(0, np.pi/3, 50).reshape([50, -1])

In [7]:
from timor.utilities.visualization import animation, MeshcatVisualizerWithAnimation
viz = MeshcatVisualizerWithAnimation()
viz.initViewer()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/


In [8]:
animation(assembly.robot, trajectory, visualizer=viz, dt=.1)
viz.viewer.jupyter_cell(height=800)